## Radio interferometría y síntesis de imágenes en astronomía - Laboratorio 1
### Vicente Mieres

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta, timezone
import juliandate as jd

## Funciones y utilidades

### Funciones principales

In [94]:
def ecef_to_enu(ecef, array_center, phi=-33.45, lamb=-70.66, rad=True, is_array=True):
  '''
  ecef in meters (xyz)
  array_center: central antenna coords
  phi: latitud
  lamb (lambda): longitude
  rad: True if phi and lambd need to be radians
  '''
  if rad:
      phi, lamb = np.deg2rad(phi), np.deg2rad(lamb)

  cphi, clam = np.cos(phi), np.cos(lamb)
  sphi, slam = np.sin(phi), np.sin(lamb)


  R = np.array([[-slam,        clam,       0],
                [-clam*sphi,   -slam*sphi, cphi],
                [clam*cphi,    slam*cphi,  sphi]])

  dxyz = np.array(ecef) - np.array(array_center)

  enu = R @ dxyz.T if is_array else R @ dxyz

  return enu


def enu_to_altaz(enu, rad=True, is_array=True):
  '''
  Transform the enu vector to altitude and azimut
  '''

  if is_array:
    E, N, U = enu[0], enu[1], enu[2]
  else:
    E, N, U = float(enu[0]), float(enu[1]), float(enu[2])

  r = np.hypot(E, N)           
  El = np.arctan2(U, r)
  A = np.arctan2(E, N)

  print("E", E.shape)
  print("N", N.shape)
  print("U", U.shape)
  print("Az", A.shape)
  print("Alt", El.shape)
  
  if rad:
      return El, A % (2*np.pi)
  else:
      A_deg = np.degrees(A) % 360.0
      El_deg = np.degrees(El)
      return El_deg, A_deg

def local_sidereal_time(longitude=-70.76, utc=None, single=True):
  """
  Calculates the local sidereal time in Radians.
  """
  if utc is None:
    now = datetime.now(timezone.utc)
  else:
    now = utc

  jd_now = jd.from_gregorian(now.year, now.month, now.day, now.hour, now.minute, now.second)
  T = (jd_now - 2451545.0) / 36525
  theta = 280.46061837 + 360.98564736629 * (jd_now - 2451545) + (0.000387933 * T * T) - (T * T * T / 38710000.0)
  deg = theta % 360 + longitude

  h,m,s = degree_to_time(deg)
  rad = np.deg2rad(deg)
  
  if single: 
    return rad 
  else: 
    return deg, rad, h, m, s

def degree_to_time(theta, is_rad=False):
  """
  Converts degrees to hours, minutes and seconds.
  """
  if is_rad:
    theta = np.rad2deg(theta)

  h = int(theta/ 15)
  m = (int)(((theta / 15) - h) * 60)
  s = ((((theta / 15) - h) * 60) - m) * 60
  return h, m, s

def Rz(theta):
  c, s = np.cos(theta), np.sin(theta)
  return np.array([[ c, -s, 0],
                   [ s, c, 0],
                   [ 0, 0, 1]])

def Ry(theta):
  c, s = np.cos(theta), np.sin(theta)
  return np.array([[ c, 0, s],
  [ 0, 1, 0],
  [-s, 0, c]])


def hor_to_eq(enu, alt, az, phi=-33.45):
    """
    Convierte un baseline ENU en coordenadas ecuatoriales (X, Y, Z),
    equivalente a baseline_to_XYZ().

    Parámetros
    ----------
    enu : array (..., 3)
        Coordenadas de baselines en el sistema local (E, N, U)
    alt, az : array o float
        Ángulos de elevación y acimut de la fuente (radianes)
    phi : float
        Latitud del observatorio (radianes)
    Retorna
    -------
    r_eq : array (..., 3)
        Coordenadas ecuatoriales (X, Y, Z)
    """
    phi = np.radians(phi)

    # Convertir a np.array y asegurar forma (...,3)
    enu = np.asarray(enu)
    if enu.shape[-1] != 3:
        raise ValueError("El array ENU debe tener forma (..., 3)")

    # Separar componentes
    E, N, U = enu[..., 0], enu[..., 1], enu[..., 2]

    # Norma del baseline
    b_norm = np.sqrt(E**2 + N**2 + U**2)

    # Calcular X, Y, Z (idéntico a baseline_to_XYZ)
    X = b_norm * ( np.sin(alt)*np.cos(phi) - np.cos(alt)*np.sin(phi)*np.cos(az) )
    Y = b_norm * ( np.sin(az)*np.cos(alt) )
    Z = b_norm * ( np.sin(alt)*np.sin(phi) + np.cos(alt)*np.cos(phi)*np.cos(az) )

    return np.stack([X, Y, Z], axis=-1)


def eq_to_uvw(H, delta, hor_coords, is_array=True):
  '''
  Return the uvw coords of an antenna o antenna array 
  '''
  ch, cd = np.cos(H), np.cos(delta)
  sh, sd = np.sin(H), np.sin(delta)

  R = np.array([[sh,        ch,      0],
                [-sd*ch,   sd*sh,   cd],
                [cd*ch,   -cd*ch,   sd]])
  
  uvw = hor_coords @ R.T if is_array else R @ hor_coords

  return uvw

def baselines(enu):
  diff = enu[:, None, :] - enu[None, :, :]  # (N, N, 3)
  # diff = enu[None, :, :] - enu[:, None, :]  # (N, N, 3)
  i, j = np.triu_indices(len(enu), k=1)
  return diff[i, j]  # (N_baselines, 3)


### Funciones secundarias

In [3]:
# utilidades 

def read_cfg_to_enu(filename, array_center=None ,phi=-33.44, lamb=-70.76, rad=True):
  '''
  Read file and return antenna config on ENU coords
  '''
  with open(filename, "r") as f:
    lines = f.readlines()

  coordsys = None 
  for line in lines:
        if line.startswith("# coordsys"):
            coordsys = line.split("=")[1].strip()
            break
  
  antennas = []
  for line in lines:
      if line.startswith("#") or not line.strip():
          continue
      parts = line.split()
      x, y, z = map(float, parts[:3])
      antennas.append([x, y, z])
  antennas = np.array(antennas)

  if coordsys == "LOC (local tangent plane)": return antennas.T
  elif coordsys == "XYZ":
     array_center = array_center if array_center is not None else antennas.mean(axis=0)
     enu_antennas = ecef_to_enu(antennas, array_center, phi, lamb, rad)
     return np.array(enu_antennas)
  else:
    raise ValueError(f"coordsys desconocido: {coordsys}")

def plot_antennas(enu_coords, labels=True, title="Configuración de Antenas"):
    """
    Muestra la configuración de antenas a partir de sus coordenadas ENU.

    Parámetros
    ----------
    enu_coords : ndarray (N,3)
        Arreglo con las coordenadas [E, N, U] de cada antena.
    labels : bool
        Si True, muestra el índice de cada antena en el gráfico.
    title : str
        Título del gráfico.
    """
    enu_coords = np.array(enu_coords)
    E, N, U = enu_coords[:,0], enu_coords[:,1], enu_coords[:,2]

    fig = plt.figure(figsize=(12,6))

    # --- Subplot 1: Vista en planta (E vs N) ---
    ax1 = fig.add_subplot(1,2,1)
    ax1.scatter(E, N, c='black', marker='o')
    if labels:
        for i, (e, n) in enumerate(zip(E, N)):
            ax1.text(e, n, str(i), fontsize=9, ha='right')
    ax1.set_xlabel("East [m]")
    ax1.set_ylabel("North [m]")
    ax1.set_title("Vista en planta (E-N)")
    ax1.grid(True)
    ax1.axis('equal')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


In [84]:
def eq_to_uvw_array(H_array, delta, r_eq):
    """
    Aplica la rotación (Ecuación 4) a todos los baselines y horas ángulo.
    r_eq: (N_baselines, 3)
    H_array: (N_H,)
    delta: float (rad)
    Retorna: uvw (N_baselines, N_H, 3)
    """
    cd, sd = np.cos(delta), np.sin(delta)
    ch, sh = np.cos(H_array), np.sin(H_array)

    # Matrices de rotación para cada H  → forma (N_H, 3, 3)
    R = np.stack([
        np.stack([sh,        ch,        np.zeros_like(H_array)], axis=-1),
        np.stack([-sd*ch,    sd*sh,     np.full_like(H_array, cd)], axis=-1),
        np.stack([cd*ch,    -cd*sh,     np.full_like(H_array, sd)], axis=-1)
    ], axis=-2)  # axis=-2 -> (N_H, 3, 3)
    
    # ✅ Ahora R.shape = (N_H, 3, 3)
    # r_eq.shape = (N_baselines, 3)
    # Queremos: uvw[b, h, i] = R[h, i, j] @ r_eq[b, j]
    uvw = np.einsum('hij,bj->bhi', R, r_eq)  # (N_baselines, N_H, 3)

    return uvw


In [95]:
# 1. Leer antenas y obtener baselines

centro = np.array([ 837.35156666, -413.38164879,  -62.61610241])

enu = read_cfg_to_enu("../antenna_arrays/alma.cycle10.9.cfg", array_center=centro)
baselines_enu = baselines(enu.T) 

print(baselines_enu.T)

# 2. Convertir a alt-az → ecuatorial
alt, az = enu_to_altaz(baselines_enu.T, rad=True)

r_eq = hor_to_eq(baselines_enu, alt, az, phi=-33.45)

r_eq


[[ 4.26241828e+02  3.39133356e+02  6.92699194e+02 ...  1.59400600e+03
   1.55460400e+03 -3.94020000e+01]
 [ 1.56133787e+02 -6.65455971e+02 -3.36111345e+02 ... -8.08219100e+03
  -9.75210200e+03 -1.66991100e+03]
 [-9.68929222e-01 -4.92549877e+00 -7.91671399e-01 ... -1.11758000e+02
  -9.80030000e+01  1.37550000e+01]]
E (903,)
N (903,)
U (903,)
Az (903,)
Alt (903,)


array([[   85.25391699,   426.2418285 ,   130.80698674],
       [ -370.9150431 ,   339.1333564 ,  -552.51962935],
       [ -185.92816738,   692.6991941 ,  -280.00389204],
       ...,
       [-4548.22421066,  1594.006     , -6681.91273069],
       [-5457.21758398,  1554.604     , -8082.81351063],
       [ -908.99337332,   -39.402     , -1400.90077993]], shape=(903, 3))